## **Modelo de Regresión**
El objetivo de este ejercicio es encontrar un modelo que permita predecir la tripulación requerida para un crucero.
Utilizaremos "Decisión Tree" y "random forest" y necesitaremos transformar una columna del tipo string en factores (Categorización)

https://scikit-learn.org/stable/modules/tree.html
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Ayudas:
1. ¿Como listar el tipo de datos de cada una de las columnas?
for i in data.columns:
  print("Columna {} tipo: {}".format(i,data[i].dtype))

2. ¿Como categorizar un conjunto de columnas?
* hacemos una copia de resguardo
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

* aplicamos label encoder a cada una de las columnas definidas en "object_cols". Las columnas se reemplazan en el conjunto de datos.

label_encoder = LabelEncoder()
object_cols = ['xxxx', 'yyyy']
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])


In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder



## Lectura de datos y exploración

Leer el fichero de datos 

In [2]:
file_path = "cruise_ship_info.csv"
data = pd.read_csv(file_path)

Listar las primeras filas del fichero para ver los datos

In [3]:
data.head()

,Ship_name,Cruise_line,Age,Tonnage,passengers,length,cabins,passenger_density,crew
0,Journey,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55
1,Quest,Azamara,6,30.277,6.94,5.94,3.55,42.64,3.55
2,Celebration,Carnival,26,47.262,14.86,7.22,7.43,31.80,6.70
3,Conquest,Carnival,11,110.000,29.74,9.53,14.88,36.99,19.10
4,Destiny,Carnival,17,101.353,26.42,8.92,13.21,38.36,10.00


¿cuantas filas y columnas tienen los datos?


In [ ]:
data.shape

(158, 9)

Explore la cantidad de registros, la media, el desvio y los cuartiles de las columnas numéricas.

In [6]:
data.describe

<bound method NDFrame.describe of        Ship_name Cruise_line  Age  Tonnage  passengers  length  cabins  \
0        Journey     Azamara    6   30.277        6.94    5.94    3.55   
1          Quest     Azamara    6   30.277        6.94    5.94    3.55   
2    Celebration    Carnival   26   47.262       14.86    7.22    7.43   
3       Conquest    Carnival   11  110.000       29.74    9.53   14.88   
4        Destiny    Carnival   17  101.353       26.42    8.92   13.21   
..           ...         ...  ...      ...         ...     ...     ...   
153       Taurus        Star   22    3.341        0.66    2.79    0.33   
154        Virgo        Star   14   76.800       19.60    8.79    9.67   
155       Spirit    Windstar   25    5.350        1.58    4.40    0.74   
156         Star    Windstar   27    5.350        1.67    4.40    0.74   
157         Surf    Windstar   23   14.745        3.08    6.17    1.56   

     passenger_density   crew  
0                42.64   3.55  
1            

Liste las columnas del dataframe indicando el tipo de datos.

In [7]:
for col in data.columns:
    print(f"Columna {col} tipo: {data[col].dtype}")

Columna Ship_name tipo: object
Columna Cruise_line tipo: object
Columna Age tipo: int64
Columna Tonnage tipo: float64
Columna passengers tipo: float64
Columna length tipo: float64
Columna cabins tipo: float64
Columna passenger_density tipo: float64
Columna crew tipo: float64


## Preparación de los datos

Defina la columna objetivo a predecir en "y"  y "X" el dataframe con las características que se utilizarán para definir el modelo. Tenga en cuenta que solo se utilizarán las columnas numéricas.

In [9]:
features = ['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density']
X = data[features]
y = data['crew']



Defina las particiones para entrenamiento y de validación con semila=1 e imprima cuantas filas utilizará para entrenamiento y cuantas para validar

In [11]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [15]:
print(train_X.shape)


(118, 6)


In [14]:
print(val_X.shape)

(40, 6)


## Definición del modelo

Defina el modelo utilizando un arbol de decisión con semilla=1

In [16]:
#Specify Model
cruise_ship_dt_model = DecisionTreeRegressor(random_state=1)


Entrene el modelo 

In [17]:
# Fit Model
cruise_ship_dt_model.fit(train_X, train_y)

DecisionTreeRegressor(random_state=1)

## Predicción y validación del modelo 

Utilice el modelo para realizar las predicciones con los datos de validación y calcule el valor MAE (mean absolute error) obtenido sin haber indicado la cantidad de hojas (max_leaf_nodes).

In [18]:
val_predictions = cruise_ship_dt_model.predict(val_X)
val_mae = mean_absolute_error(val_y, val_predictions)
print("Validation MAE when not specifying max_leaf_nodes %f" %val_mae)

Validation MAE when not specifying max_leaf_nodes 0.503000


Defina la función "get_mae" que le permite calcular valores MAE para diferentes arbóles de decisión con diferentes números de hojas

In [19]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

Utilice la función get_mae para calcular MAE para los tamaños de 10,50,100,200,500,1000

In [20]:
# Calcular MAE para diferentes valores de max_leaf_nodes
for max_leaf_nodes in [10, 50, 100, 200, 500, 1000]:
	mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
	print("Max leaf nodes: %d \t\t Mean Absolute Error: %f" %(max_leaf_nodes, mae))

Max leaf nodes: 10 		 Mean Absolute Error: 0.768456
Max leaf nodes: 50 		 Mean Absolute Error: 0.547230
Max leaf nodes: 100 		 Mean Absolute Error: 0.481750
Max leaf nodes: 200 		 Mean Absolute Error: 0.481750
Max leaf nodes: 500 		 Mean Absolute Error: 0.481750
Max leaf nodes: 1000 		 Mean Absolute Error: 0.481750


¿Cuál es el valor optimo de "max_leaf_nodes" a utilizar para este modelo?

In [21]:
max_leaf_nodes = 100

In [22]:
assert max_leaf_nodes==100,"Error en max_leaf_nodes"

Calcule el vector de predicciones utilizando el modelo con la cantidad de hojas optimo y semilla 1 utilizando ahora todos los datos. Calcule el mae final.

In [23]:
model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
model.fit(X, y)
pred_tot = model.predict(X)
mae = mean_absolute_error(y, pred_tot)
print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %f" %(max_leaf_nodes, mae))

Max leaf nodes: 100  		 Mean Absolute Error:  0.007004


¿Que porcentaje de error obtengo respecto a la media de los valores a predecir?

In [24]:
print((mae / y.mean()) * 100)

0.08986476976213031


In [25]:
assert round((mae / y.mean())*100)==0,"Error en resultado de porcentaje de error"

Explore al menos 10 valores calculados con la predicción y compare con los valore originales

In [26]:
# Mostrar los primeros 10 valores predichos y los valores originales
for i, (pred, real) in enumerate(zip(pred_tot[:10], y[:10])):
	print(f"Predicción: {pred:.2f} \t Valor real: {real:.2f}")

Predicción: 3.53 	 Valor real: 3.55
Predicción: 3.53 	 Valor real: 3.55
Predicción: 6.65 	 Valor real: 6.70
Predicción: 19.10 	 Valor real: 19.10
Predicción: 10.00 	 Valor real: 10.00
Predicción: 9.20 	 Valor real: 9.20
Predicción: 9.20 	 Valor real: 9.20
Predicción: 9.20 	 Valor real: 9.20
Predicción: 9.20 	 Valor real: 9.20
Predicción: 11.50 	 Valor real: 11.50


## Random Forest 

Utilice ahora un modelo RandomForest con semilla=1 y calcule el MAE para los datos de validación

In [27]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
val_predict_rf = forest_model.predict(val_X)
mae_val_rf = mean_absolute_error(val_y, val_predict_rf)
print("Mae Random forest con datos de validación=%f" % (mae_val_rf))


Mae Random forest con datos de validación=0.432155


 Utilice todos los datos

In [28]:
forest_model.fit(X, y)
predict_rf_tot = forest_model.predict(X)
mae_tot_rf = mean_absolute_error(y, predict_rf_tot)
print("Mae Random forest con todos los datos=%f" % (mae_tot_rf))


Mae Random forest con todos los datos=0.198572


¿Que porcentaje de error obtengo respecto a la media de los valores a predecir utilizando random_forest?

In [29]:
print((mae_tot_rf / y.mean()) * 100)

2.5477003280605484


In [30]:
assert round((mae_tot_rf / y.mean())*100)==3,"Error en resultado de porcentaje de error"

## Categorización de variables

redefinir las columnas "features" y los datos X. Volver a particionar los datos incluyendo la nueva columna a factorizar

In [31]:
features=["Cruise_line", "Age", "Tonnage","passengers",'length','cabins','passenger_density']
X = data[features]

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1)

Definir la funcion label_encoder, la lista de las columnas a transformar y aplicar la transpormación.

In [33]:
#hacemos una copia de resguardo
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
object_cols = ['Cruise_line']
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])



Explorar el vector X de entrenamiento antes y despues de aplicar la categorización. Utilice set, unique() y nunique()

In [34]:
set(X['Cruise_line'])

{'Azamara',
 'Carnival',
 'Celebrity',
 'Costa',
 'Crystal',
 'Cunard',
 'Disney',
 'Holland_American',
 'MSC',
 'Norwegian',
 'Oceania',
 'Orient',
 'P&O',
 'Princess',
 'Regent_Seven_Seas',
 'Royal_Caribbean',
 'Seabourn',
 'Silversea',
 'Star',
 'Windstar'}

In [35]:
print("Cantidad de Unique values in Cruise_line column in training data:", X_train['Cruise_line'].nunique())

Cantidad de Unique values in Cruise_line column in training data: 20


In [36]:
print(label_X_train.head)

<bound method NDFrame.head of      Cruise_line  Age  Tonnage  passengers  length  cabins  passenger_density
56             7   10   81.769       18.48    9.59    9.24              44.25
136           15   16   78.491       24.35    9.15   10.00              32.23
132           15   22   73.941       27.44    8.80   11.77              30.94
12             1   18   70.367       20.52    8.55   10.20              34.29
54             7   20   55.451       12.64    7.19    6.32              43.87
..           ...  ...      ...         ...     ...     ...                ...
133           15   11  138.000       31.14   10.20   15.57              44.32
137           15   10   90.090       25.01    9.62   10.50              36.02
72             8   36   16.852        9.52    5.41    3.83              17.70
140           15   15   78.491       24.35    9.15   10.00              32.23
37             3   27   53.872       14.94    7.98    7.67              36.06

[118 rows x 7 columns]>


In [37]:
print("Unique values in Cruise_line luego de aplicar la codificación de label:", label_X_train['Cruise_line'].unique())

Unique values in Cruise_line luego de aplicar la codificación de label: [ 7 15  1 13  2  8 14  9  5 11  4  3 17  0 10 18 12 19  6 16]


Defina un nuevo modelo utilizando con la nueva característica e indique los nuevos valores MAE con los datos de validación utilizando Random Forest y Decision Tree. ¿Ha conseguido disminuir el error?

In [38]:
def score_dataset_RandomForest(X_train, X_valid, y_train, y_valid,n_estimators=100):
    model = RandomForestRegressor(n_estimators=100, random_state=1)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

def score_dataset_DecisionTree(X_train, X_valid, y_train, y_valid,max_leaf_nodes=100):
    model=DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)


In [39]:
print("MAE incluyendo Cruise Line utilizando Random Forest:") 
print(score_dataset_RandomForest(label_X_train, label_X_valid, y_train, y_valid))

MAE incluyendo Cruise Line utilizando Random Forest:
0.3761275000000009


In [40]:
print("MAE incluyendo Cruise Line utilizando Decision Tres:") 
print(score_dataset_DecisionTree(label_X_train, label_X_valid, y_train, y_valid))

MAE incluyendo Cruise Line utilizando Decision Tres:
0.45899999999999996
